<a href="https://colab.research.google.com/github/cleberjamaral/autoOrgDesignProject/blob/master/GoOrg4Prod_structure_choosing_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [125]:
import pandas as pd
import numpy as np
import altair as alt # for charts
from scipy import stats, spatial

In [126]:
bound_df = pd.read_csv("output/statistics/Feed_production_line_binding.csv","\t")
#skills = ["db_access","move","lift","pnp"]
skills = ["lift","move","pnp"]

In [127]:
bound_df.head()

,id,nPosit,%Idle+,%Effic,%Geral,%Speci,%Tall,%Flat,%Feasi,Levels,...,rWL,bDL,rDL,%WL+,%DL+,pTree,bgTree,skTree,agents,matche
0,1,1,46%,54%,100%,0%,0%,100%,0%,1,...,13.0,16.0,0.0,100%,0%,"[G{[GetBox, MoveBox, PlaceBox]}]","[PlaceBox, GetBox, MoveBox]","[[pnp, move, lift]]","[pp, bt, ie]",NaN
1,2,2,73%,27%,25%,75%,0%,100%,50%,1,...,13.0,16.0,8.0,100%,50%,"[G{[GetBox, MoveBox]}, G{[PlaceBox]}]","[PlaceBox, GetBox, MoveBox]","[[move, lift], [pnp]]","[pp, bt, ie]",p1=pp
2,3,2,73%,27%,25%,75%,0%,100%,50%,1,...,13.0,16.0,16.0,100%,100%,"[G{[GetBox, PlaceBox]}, G{[MoveBox]}]","[PlaceBox, GetBox, MoveBox]","[[move], [pnp, lift]]","[pp, bt, ie]",p1=bt
3,4,2,73%,27%,25%,75%,0%,100%,50%,1,...,13.0,16.0,8.0,100%,50%,"[G{[GetBox]}, G{[MoveBox, PlaceBox]}]","[PlaceBox, GetBox, MoveBox]","[[lift], [pnp, move]]","[pp, bt, ie]",p1=ie
4,5,2,73%,27%,25%,75%,50%,50%,50%,2,...,13.0,16.0,8.0,100%,50%,"[G{[GetBox, MoveBox]}, G{[PlaceBox]}^G{[GetBox...","[PlaceBox, GetBox, MoveBox]","[[move, lift], [pnp]]","[pp, bt, ie]",p1=pp


In [128]:
# Plotting configurations
CHART_WIDTH = 600
CHART_HEIGHT = 400

In [129]:
#bound_df.dtypes

In [130]:
# Generality similarity
dfsimilarity = pd.DataFrame(columns=["id_a", "id_b", "similarity"])
dfsimilarity.columns = dfsimilarity.columns.map(str)
new_i = -1

# Process correlations for table with 3 columns
for i in range(len(bound_df["id"])):
    for j in range(len(bound_df["id"])):
        # df.values[row, column] = value
        if (new_i != i):
            new_i = i
            #print("Processing line ",i)
        dfsimilarity.loc[(i*len(bound_df["id"]))+j] = (
            bound_df["id"][i], 
            bound_df["id"][j], 
            100-abs(round((float(bound_df['%Geral'][i].rstrip('%'))-float(bound_df['%Geral'][j].rstrip('%')))))
         )

In [131]:
dfplot = pd.DataFrame({'x': dfsimilarity["id_a"].ravel(),
                   'y': dfsimilarity["id_b"].ravel(),
                   'Similarity': dfsimilarity["similarity"].ravel()})
#alt.data_transformers.disable_max_rows()
chart = alt.Chart(dfplot).mark_rect().encode(
    x=alt.X('x:O', axis=alt.Axis(title="")),
    y=alt.Y('y:O', axis=alt.Axis(title=""), sort="descending"),
    color='Similarity:Q'
).properties(
    title=["Generality Similarity Comparison"]
)

text = chart.mark_text(baseline='middle').encode(
    text='Similarity:Q',
    color=alt.condition(
        alt.datum.Similarity < 70,
        alt.value('black'),
        alt.value('white')
    ),
    size=alt.value(10),
    opacity=alt.value(0.85)
)

# Draw the chart
plot = chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + text
plot

alt.LayerChart(...)

In [132]:
# Efficiency similarity
dfsimilarity = pd.DataFrame(columns=["id_a", "id_b", "similarity"])
dfsimilarity.columns = dfsimilarity.columns.map(str)
new_i = -1

# Process correlations for table with 3 columns
for i in range(len(bound_df["id"])):
    for j in range(len(bound_df["id"])):
        # df.values[row, column] = value
        if (new_i != i):
            new_i = i
            #print("Processing line ",i)
        dfsimilarity.loc[(i*len(bound_df["id"]))+j] = (
            bound_df["id"][i], 
            bound_df["id"][j], 
            100-abs(round((float(bound_df['%Effic'][i].rstrip('%'))-float(bound_df['%Effic'][j].rstrip('%')))))
         )

In [133]:
dfplot = pd.DataFrame({'x': dfsimilarity["id_a"].ravel(),
                   'y': dfsimilarity["id_b"].ravel(),
                   'Similarity': dfsimilarity["similarity"].ravel()})
#alt.data_transformers.disable_max_rows()
chart = alt.Chart(dfplot).mark_rect().encode(
    x=alt.X('x:O', axis=alt.Axis(title="")),
    y=alt.Y('y:O', axis=alt.Axis(title=""), sort="descending"),
    color='Similarity:Q'
).properties(
    title=["Efficiency Similarity Comparison"]
)

text = chart.mark_text(baseline='middle').encode(
    text='Similarity:Q',
    color=alt.condition(
        alt.datum.Similarity < 80,
        alt.value('black'),
        alt.value('white')
    ),
    size=alt.value(10),
    opacity=alt.value(0.85)
)

# Draw the chart
plot = chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + text
plot

alt.LayerChart(...)

In [134]:
# Flatness similarity
dfsimilarity = pd.DataFrame(columns=["id_a", "id_b", "similarity"])
dfsimilarity.columns = dfsimilarity.columns.map(str)
new_i = -1

# Process correlations for table with 3 columns
for i in range(len(bound_df["id"])):
    for j in range(len(bound_df["id"])):
        dfsimilarity.loc[(i*len(bound_df["id"]))+j] = (
            bound_df["id"][i], 
            bound_df["id"][j], 
            100-abs(round((float(bound_df['%Flat'][i].rstrip('%'))-float(bound_df['%Flat'][j].rstrip('%')))))
         )

In [135]:
dfplot = pd.DataFrame({'x': dfsimilarity["id_a"].ravel(),
                   'y': dfsimilarity["id_b"].ravel(),
                   'Similarity': dfsimilarity["similarity"].ravel()})
#alt.data_transformers.disable_max_rows()
chart = alt.Chart(dfplot).mark_rect().encode(
    x=alt.X('x:O', axis=alt.Axis(title="")),
    y=alt.Y('y:O', axis=alt.Axis(title=""), sort="descending"),
    color='Similarity:Q'
).properties(
    title=["Height Similarity Comparison"]
)

text = chart.mark_text(baseline='middle').encode(
    text='Similarity:Q',
    color=alt.condition(
        alt.datum.Similarity < 70,
        alt.value('black'),
        alt.value('white')
    ),
    size=alt.value(10),
    opacity=alt.value(0.85)
)

# Draw the chart
plot = chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + text
plot

alt.LayerChart(...)

In [136]:
def powerset(s):
    P = []
    x = len(s)
    for i in range(1 << x):
        P.append([s[j] for j in range(x) if (i & (1 << j))])
    return P

In [137]:
def get_organisations(array_str):
    organisations = []
    for i in range(0,len(array_str)):
        positions = []
        temp = str(array_str[i][1:-1]).split(", [")
        for j in range(0,len(temp)):
            skills = str(temp[j]).strip("[]").split(",")
            for s in range(0,len(skills)):
                skills[s] = skills[s].strip()
            positions.append(skills)
        organisations.append(positions)
    return organisations
orgs = get_organisations(bound_df["skTree"])

In [149]:
# For each organisation, check which kinds of agent it has
all_agent_kinds = powerset(skills)
all_kinds = []
orgs = get_organisations(bound_df["skTree"])

print(all_agent_kinds)
def check_all_kinds_of_agents_in_orgs():
    for o in range(0,len(orgs)):
        kinds = []
        for k in range(0,len(all_agent_kinds)):
            for p in range(0,len(orgs[o])):
                if set(all_agent_kinds[k]) == set(orgs[o][p]):
                    kinds.append(k)
                    print(o+1,set(all_agent_kinds[k]),">>",set(orgs[o][p]),"::",kinds)
        all_kinds.append(kinds)
        
check_all_kinds_of_agents_in_orgs()

[[], ['lift'], ['move'], ['lift', 'move'], ['pnp'], ['lift', 'pnp'], ['move', 'pnp'], ['lift', 'move', 'pnp']]
1 {'lift', 'pnp', 'move'} >> {'lift', 'pnp', 'move'} :: [7]
2 {'lift', 'move'} >> {'lift', 'move'} :: [3]
2 {'pnp'} >> {'pnp'} :: [3, 4]
3 {'move'} >> {'move'} :: [2]
3 {'lift', 'pnp'} >> {'lift', 'pnp'} :: [2, 5]
4 {'lift'} >> {'lift'} :: [1]
4 {'pnp', 'move'} >> {'pnp', 'move'} :: [1, 6]
5 {'lift', 'move'} >> {'lift', 'move'} :: [3]
5 {'pnp'} >> {'pnp'} :: [3, 4]
6 {'move'} >> {'move'} :: [2]
6 {'lift', 'pnp'} >> {'lift', 'pnp'} :: [2, 5]
7 {'lift'} >> {'lift'} :: [1]
7 {'pnp', 'move'} >> {'pnp', 'move'} :: [1, 6]
8 {'lift'} >> {'lift'} :: [1]
8 {'pnp', 'move'} >> {'pnp', 'move'} :: [1, 6]
9 {'lift', 'move'} >> {'lift', 'move'} :: [3]
9 {'pnp'} >> {'pnp'} :: [3, 4]
10 {'move'} >> {'move'} :: [2]
10 {'lift', 'pnp'} >> {'lift', 'pnp'} :: [2, 5]
11 {'lift'} >> {'lift'} :: [1]
11 {'move'} >> {'move'} :: [1, 2]
11 {'pnp'} >> {'pnp'} :: [1, 2, 4]
12 {'lift'} >> {'lift'} :: [1]
12 

In [159]:
# Kinds of agents similarity
dfsimilarity = pd.DataFrame(columns=["id_a", "id_b", "similarity"])
dfsimilarity.columns = dfsimilarity.columns.map(str)
#maxLength = max(len(x) for x in all_kinds)

# Process correlations for table with 3 columns
for i in range(len(bound_df["id"])):
    for j in range(len(bound_df["id"])):
        maxLength = len(set(all_kinds[i] + all_kinds[j]))
        dfsimilarity.loc[(i*len(bound_df["id"]))+j] = (
            bound_df["id"][i], 
            bound_df["id"][j],
            round(100*(
                max(maxLength,1)-
                len(
                    list(set(all_kinds[i]) - set(all_kinds[j])) + 
                    list(set(all_kinds[j]) - set(all_kinds[i]))
                )
            )/max(maxLength,1))
        )        

In [158]:
dfplot = pd.DataFrame({'x': dfsimilarity["id_a"].ravel(),
                   'y': dfsimilarity["id_b"].ravel(),
                   'Similarity': dfsimilarity["similarity"].ravel()})
#alt.data_transformers.disable_max_rows()
chart = alt.Chart(dfplot).mark_rect().encode(
    x=alt.X('x:O', axis=alt.Axis(title="")),
    y=alt.Y('y:O', axis=alt.Axis(title=""), sort="descending"),
    color='Similarity:Q'
).properties(
    title=["Kind of Agents Similarity Comparison"]
)

text = chart.mark_text(baseline='middle').encode(
    text='Similarity:Q',
    color=alt.condition(
        alt.datum.Similarity < 70,
        alt.value('black'),
        alt.value('white')
    ),
    size=alt.value(10),
    opacity=alt.value(0.85)
)

# Draw the chart
plot = chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + text
plot

alt.LayerChart(...)

In [156]:
# Switching structure cost
dfsimilarity = pd.DataFrame(columns=["id_a", "id_b", "cost"])
dfsimilarity.columns = dfsimilarity.columns.map(str)
#print(len(all_kinds),all_kinds)
# Process correlations for table with 3 columns
for i in range(len(bound_df["id"])):
    for j in range(len(bound_df["id"])):
        dfsimilarity.loc[(i*len(bound_df["id"]))+j] = (
            bound_df["id"][i], 
            bound_df["id"][j],
            1*len(list(set(all_kinds[j]) - set(all_kinds[i])))
        )
        #print(set(all_kinds[j]),set(all_kinds[i]),list(set(all_kinds[j]) - set(all_kinds[i])))
        

In [153]:
dfplot = pd.DataFrame({'x': dfsimilarity["id_a"].ravel(),
                   'y': dfsimilarity["id_b"].ravel(),
                   'Cost': dfsimilarity["cost"].ravel()})
#alt.data_transformers.disable_max_rows()
chart = alt.Chart(dfplot).mark_rect().encode(
    x=alt.X('x:O', axis=alt.Axis(title="")),
    y=alt.Y('y:O', axis=alt.Axis(title=""), sort="descending"),
    color='Cost:Q'
).properties(
    title=["Switching Structure Cost"]
)

text = chart.mark_text(baseline='middle').encode(
    text='Cost:Q',
    color=alt.condition(
        alt.datum.Cost < 3,
        alt.value('black'),
        alt.value('white')
    ),
    size=alt.value(10),
    opacity=alt.value(0.85)
)

# Draw the chart
plot = chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + text
plot

alt.LayerChart(...)

In [154]:
# Total Qualification Change
dfsimilarity = pd.DataFrame(columns=["id_a", "id_b", "change"])
dfsimilarity.columns = dfsimilarity.columns.map(str)

# Process correlations for table with 3 columns
for i in range(len(bound_df["id"])):
    for j in range(len(bound_df["id"])):
        dfsimilarity.loc[(i*len(bound_df["id"]))+j] = (
            bound_df["id"][i], 
            bound_df["id"][j],
            len(all_skills[j]) - len(all_skills[i])
        )
        #print(bound_df["id"][i],bound_df["id"][j],(len(all_skills[j]) - len(all_skills[i])),all_skills[i],all_skills[j])        

In [155]:
dfplot = pd.DataFrame({'x': dfsimilarity["id_a"].ravel(),
                   'y': dfsimilarity["id_b"].ravel(),
                   'Change': dfsimilarity["change"].ravel()})
#alt.data_transformers.disable_max_rows()
chart = alt.Chart(dfplot).mark_rect().encode(
    x=alt.X('x:O', axis=alt.Axis(title="")),
    y=alt.Y('y:O', axis=alt.Axis(title=""), sort="descending"),
    color='Change:Q'
).properties(
    title=["Total Qualification Changes"]
)

text = chart.mark_text(baseline='middle').encode(
    text='Change:Q',
    color=alt.condition(
        alt.datum.Change < 3,
        alt.value('black'),
        alt.value('white')
    ),
    size=alt.value(10),
    opacity=alt.value(0.85)
)

# Draw the chart
plot = chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + text
plot

alt.LayerChart(...)

In [86]:
# Sum of similarities
dfsimilarity = pd.DataFrame(columns=["id_a", "id_b", "similarity"])
dfsimilarity.columns = dfsimilarity.columns.map(str)
new_i = -1

# Process correlations for table with 3 columns
for i in range(len(bound_df["id"])):
    for j in range(len(bound_df["id"])):
        dfsimilarity.loc[(i*len(bound_df["id"]))+j] = (
            bound_df["id"][i],
            bound_df["id"][j],
            round((
            100-abs(round((float(bound_df['%Geral'][i].rstrip('%'))-float(bound_df['%Geral'][j].rstrip('%')))))+
            100-abs(round((float(bound_df['%Effic'][i].rstrip('%'))-float(bound_df['%Effic'][j].rstrip('%')))))+
            100-abs(round((float(bound_df['%Flat'][i].rstrip('%'))-float(bound_df['%Flat'][j].rstrip('%')))))+
            round(100*(max(all_kinds)[0]-len(list(set(all_kinds[i]) - set(all_kinds[j])) + list(set(all_kinds[j]) - set(all_kinds[i]))))/max(all_kinds)[0])
            )/4)
         )

In [87]:
dfplot = pd.DataFrame({'x': dfsimilarity["id_a"].ravel(),
                   'y': dfsimilarity["id_b"].ravel(),
                   'Similarity': dfsimilarity["similarity"].ravel()})
#alt.data_transformers.disable_max_rows()
chart = alt.Chart(dfplot).mark_rect().encode(
    x=alt.X('x:O', axis=alt.Axis(title="")),
    y=alt.Y('y:O', axis=alt.Axis(title=""), sort="descending"),
    color='Similarity:Q'
).properties(
    title=["Sum of Similarities Comparison"]
)

text = chart.mark_text(baseline='middle').encode(
    text='Similarity:Q',
    color=alt.condition(
        alt.datum.Similarity < 70,
        alt.value('black'),
        alt.value('white')
    ),
    size=alt.value(10),
    opacity=alt.value(0.85)
)

# Draw the chart
plot = chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + text
plot

alt.LayerChart(...)